# 05 — Error Handling + Logging (1DownLabs)

This notebook covers:
- try/except/else/finally
- common error types
- defensive input validation
- logging (INFO/WARNING/ERROR)
- building a safe CSV loading function
- practice exercises + mini pipeline

In [1]:
import sys
from pathlib import Path

project_root = Path("..").resolve()
sys.path.append(str(project_root))

raw_dir = project_root / "data" / "raw"
processed_dir = project_root / "data" / "processed"
raw_dir.mkdir(parents=True, exist_ok=True)
processed_dir.mkdir(parents=True, exist_ok=True)

## Common error types

- FileNotFoundError: file path is wrong
- KeyError: missing DataFrame column / dict key
- ValueError: invalid conversion (e.g., "abc" to int)
- TypeError: wrong type (e.g., adding str + int)
- ZeroDivisionError: division by zero

In [5]:
# try , except Basic

try:
    x = 10 / 0
    print (x)
except ZeroDivisionError as e:
    print("Caught error:", e)

Caught error: division by zero


In [6]:
# try/except/else/finally

def safe_divide(a, b):
    try:
        result = a / b
    except ZeroDivisionError:
        return None
    else:
        return result
    finally:
        # Always runs
        pass

print(safe_divide(10, 2))
print(safe_divide(10, 0))

5.0
None


In [7]:
# 5.1 Create a logger (Code)

import logging

logger = logging.getLogger("1downlabs")
logger.setLevel(logging.INFO)

if not logger.handlers:
    handler = logging.StreamHandler()
    formatter = logging.Formatter("%(levelname)s | %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Logger is ready")

INFO | Logger is ready


In [9]:
# Logging levels demo (Code)

logger.info("This is an INFO message")
logger.warning("This is a WARNING message")
logger.error("This is an ERROR message")

INFO | This is an INFO message
WARNING | This is a WARNING message
ERROR | This is an ERROR message


Build robust helper functions (BI/DS style)

In [10]:
# 6.1 Validate columns (Code)

def validate_columns(df, required_cols):
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise KeyError(f"Missing required columns: {missing}")
    return True

In [11]:
# Safe CSV loader with logging (Code)

import pandas as pd

def safe_read_csv(path, required_cols=None):
    path = Path(path)

    if not path.exists():
        logger.error(f"File not found: {path}")
        return None

    try:
        df = pd.read_csv(path)
        logger.info(f"Loaded CSV: {path.name} | shape={df.shape}")
    except Exception as e:
        logger.error(f"Failed to read CSV: {e}")
        return None

    if required_cols:
        try:
            validate_columns(df, required_cols)
            logger.info("Column validation passed")
        except KeyError as e:
            logger.error(str(e))
            return None

    return df


In [12]:
sample_path = raw_dir / "sales_sample.csv"
df = safe_read_csv(sample_path, required_cols=["date", "region", "sales", "orders"])
df.head() if df is not None else df


INFO | Loaded CSV: sales_sample.csv | shape=(6, 4)
INFO | Column validation passed


,date,region,sales,orders
0,2026-01-01,East,1200,45
1,2026-01-01,West,900,35
2,2026-01-02,East,1400,52
3,2026-01-02,West,800,30
4,2026-01-03,East,1100,40


In [13]:
'''Mini “safe pipeline” function

This mimics real pipelines: load → validate → transform → save.'''


def build_sales_summary(input_csv, output_csv):
    df = safe_read_csv(input_csv, required_cols=["date", "region", "sales", "orders"])
    if df is None:
        return False

    # Transform safely
    try:
        df["sales"] = pd.to_numeric(df["sales"])
        df["orders"] = pd.to_numeric(df["orders"])
        df["aov"] = df["sales"] / df["orders"]
    except Exception as e:
        logger.error(f"Transform failed: {e}")
        return False

    summary = (
        df.groupby("region")
          .agg(total_sales=("sales", "sum"),
               total_orders=("orders", "sum"),
               avg_aov=("aov", "mean"))
          .reset_index()
    )

    try:
        summary.to_csv(output_csv, index=False)
        logger.info(f"Saved output: {output_csv}")
    except Exception as e:
        logger.error(f"Save failed: {e}")
        return False

    return True



In [14]:
out_path = processed_dir / "sales_summary_safe.csv"
ok = build_sales_summary(raw_dir / "sales_sample.csv", out_path)
ok, out_path.exists()

INFO | Loaded CSV: sales_sample.csv | shape=(6, 4)
INFO | Column validation passed
INFO | Saved output: /Users/arpitshukla/1DownLabs/data-science/python-foundations/data/processed/sales_summary_safe.csv


(True, True)

In [18]:
'''Practice 1 — Safe integer conversion

Write safe_int(x) that:

returns int(x) if possible

returns None if conversion fails

logs a warning when it fails'''


def safe_int(x):
    
    try:
        return int(x)
    
    except (ValueError, TypeError) as e:
        logger.warning(f"safe_int failed for value '{x}': {e}")
        return None
 

print(safe_int("2020"))
print(safe_int("abc"))

WARNING | safe_int failed for value 'abc': invalid literal for int() with base 10: 'abc'


2020
None


In [23]:
'''Write get_value(d, key, default=None) that:

returns d[key] if present

returns default if missing

logs a warning if missing'''




def get_value(d, key, default=None):
    try:
        for key in d:
            return d[key]
    except (ValueError, TypeError) as e:
        logger.warning(f"safe_int failed for value '{key}': {e}")
        return default 


        




row = {"sales": 1000, "orders": 50}

get_value(row, key = "orders")

1000